# sample generator
This notebook is for creating the random sample .txt files for the experiment.

In [2]:
import pandas as pd
import numpy as np

 ## Contents
 1. [Create separate file with contexts](#Create-separate-file-with-contexts)
 2. [Create separate file without contexts](#Create-separate-file-without-contexts)
 3. [Creating the files for the experiment](#Creating-files-for-the-experiment)
 4. [Automate Paraphrase Generator](#Automate-pharaphrase-generator)


In [3]:
# import the database file from the TGrep2 searching
df = pd.read_csv("../results/swbd.tab", sep='\t', engine='python')

In [4]:
# This makes the display show more info
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Creating the files for the experiment

## Constrain dataset
for experimental mock-up

First we have to remove the questions that we don;t want to include:
1. non-embedded or root questions
2. no degree questions
3. no identity questions
4. generally only monomorphemic wh-phrases
5. only who-, where-, and how-questions

In [5]:
df_root = df[df.QuestionType=="root"]
df_emb = df[df.QuestionType=="embedded"]

In [6]:
critical = df[(df['QuestionType'] == 'root') # only root questions
              & 
              (df['DegreeQ'] == 'no' ) # no degree questions
              &
              (df['IdentityQ'] == "no") # no identity questions
              &
              (df['Wh'].isin(['how','How','where','Where','who','Who'])) # just these wh-words
              &
              (df['WhPhaseType'] == 'monomorphemic') # monomorphic wh only (might get anything not go by degQ)
             ]

In [10]:
(len(df_root)+len(df_emb))/len(df)*100

46.44572997352682

In [7]:
# Percentage of total dataset
len(critical)/len(df_root)*100

19.488074461896453

In [11]:
len(critical)/(len(df_root)+len(df_emb))*100

7.071986489339244

In [14]:
len(critical)/len(df)*100

3.2846357486028044

In [15]:
len(critical)

335

Notes from Judith meeting 11/5:
15 how , 10 where, 5 who

randomly divide the gru
each list should get about 30 judgements

but pilot first with 10 items to test different dependent measures


In [7]:
critical.pivot_table(index=['Wh'], values="Sentence", aggfunc=len).groupby(["Wh"]).Sentence.transform(lambda x: x/len(critical)).reset_index()

,Wh,Sentence
0,how,0.549254
1,where,0.289552
2,who,0.161194


In [9]:
40/335

0.11940298507462686

should contain about 3 modal in each batch
10 lists of 30
1 list of 30

balance so that there are 3 modals 


QUESTION PHRASING
avoid using 'paraphrase'
1. based on the question in red, which of the following does the speaker want to know?
2. "none of the above"
3. ellipsis before the 
4. practice examples that are fake from the literature clear MS/MA answers....
4. get rid of trace

should be a way to highlight just the question, and not the entire sentence
"how can you tell where it's going to come down at"


is there a way to reorder the elements of the question to create the paraphrase
strip the "do" and insert "that"


rhetorical questions: "so my family may say that on occasion, who knows"

Piloting
10 items that will be repiloted for each different testing 

vary between subjects the order of the paraphrases
sample randomly at the beginning of the experiment what the order is
but keep it consistent in the experiment and change between subjects


By monday post on proliferate*



look at the lab wifi for pre-registration on OSF

get R going....

In [10]:
len(critical)

335

## Paraphrase Generator
this should take as input the entire constrained dataframe from the above section, and then generate the paraphrases

For Who questions: Who is a person...? / Who is some person...? / "Who is every person..." / "Who is the person..."

In [11]:
# read in df with contexts
cntxts = pd.read_csv("swbd_contexts.csv")

In [12]:
cntxts = cntxts.drop(columns="FollowingContext")

In [13]:
# get the indixes from critical
crit_index = critical.Item_ID

In [14]:
# subset to the items that are just the ones filtered in the previos section

# otherwise, if using the database file with contexts directly in there, then this step
# is not necessary
df_valid = cntxts[cntxts["TGrepID"].isin(set(crit_index))]

In [15]:
who = df_valid[df_valid["EntireSentence"].str.contains("Who|who")]
where = df_valid[df_valid["EntireSentence"].str.contains("Where|where")]
how = df_valid[df_valid["EntireSentence"].str.contains("How|how")]

In [16]:
df_valid.head()

,TGrepID,EntireSentence,PreceedingContext
95,1721:4,how do you feel *t*-1 about rap music?,"###there's such a wide selection,### i think 0 i like a lot, i like a little bit of a lot of different types of music.###you know, i, i, i like music that *t*-1 is, that i feel###if it is performed *-1 correctly or if it's done *-2 right, or if the version is done *-3 right, i like it,###speakera15.###yeah.###speakerb16.###but, if, if it's not *?*, then i won't *?*.### i, i really don't,###speakera17."
189,2936:24,"oh, rea-, where *ich*-2 did you go to school in indiana *t*-1?","###speakerb58.###so, that, uh, you, you became accustomed to it, i guess.###speakera59.###uh-huh.###speakerb60.###but, uh, otherwise w-, as i said, we have had, uh, a relatively mild winter, * speaking for this area of the country.###speakera61.###uh-huh.###speakerb62.###speakera63."
246,4134:4,"how did you hear,","###do you work for t i?###speakera90.###no,###no,###i work for g t e. in maryland,###speakerb91.###really?###speakera92.###yes.###speakerb93."
247,4140:4,how did i hear about it *t*-1?,"###really?###speakera92.###yes.###speakerb93.###how did you hear,###oh we're not even supposed *-1 to be talking about this though, are we *?*?###speakera94.###i guess not.###speakerb95.###speakera96."
256,4270:11,"so, and how do you feel about it *t*-1?.","###yeah,###so. i'm, i'm an avid aerobics, uh, person and just very, very athletic.###speakera9.###uh-huh.###speakerb10.###enjoying a th-, you know, uh, anything dealing with sports.###speakera11.###uh-huh,###uh-huh.###speakerb12."


In [17]:
who["AResponse"] = "Who is a person...?"
who["SomeResponse"] = "Who is some person...?"
who["AllResponse"] = "Who is every person...?"
who["TheResponse"] = "Who is the person...?"


where["AResponse"] = "What is a place...?"
where["SomeResponse"] = "What is some place...?"
where["AllResponse"] = "What is every place...?"
where["TheResponse"] = "What is the place...?"


how["AResponse"] = "What is a way...?"
how["SomeResponse"] = "What is some way...?"
how["AllResponse"] = "What is every way...?"
how["TheResponse"] = "What is the way...?"

<ipython-input-17-a0fe390ba77f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["AResponse"] = "Who is a person...?"
<ipython-input-17-a0fe390ba77f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  who["SomeResponse"] = "Who is some person...?"
<ipython-input-17-a0fe390ba77f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [18]:
df_final = pd.concat([who,where,how])

There should be a way to automate that whole thing in one definitieion, but im failing to do that correctly

In [19]:
# def paramap(data):
#     if data["EntireSentence"].str.contains("who|Who"):
#         data["AResponse"] == "Who is a person?"
#         data["SomeResponse"] == "Who is some person?"
#         data["AllResponse"] == "Who is every person?"
#         data["TheResponse"] == "Who is the person?"
    #         return
    #     if df["EntireSentence"].str.contains(["where|Where"]):
    #         return "places"
    #     if df["EntireSentence"].str.contains(["how|How"]):
    #         return "ways"

In [20]:
# df_para = df_valid.apply(paramap)

## Create randomly sampled files

## Pilot Samples

### Pilot Ex 01

In [21]:
pilot = df_final.sample(10, random_state=333)

In [22]:
# pilot.to_csv("../../experiments/clean_corpus/pilot1.txt",header=True,sep="\t",index=False)

### Pilot Ex 02
This design uses sliders rather than radio buttons, and is not a forced choice.

Stimuli are the same as Pilot Ex01, but with rhetorical questions replaced.

In [23]:
# rhetorical questions to remove
# 5928, 2328
pilot.EntireSentence

3582                                                    how are your kids handling it *t*-1?
3511                                                                        who *t*-1 knows.
2178                           well how do you think 0 the royals are going *-1 to do *t*-2?
5928    but, uh, who *t*-1 knows what the future will hold *t*-2. uh, as far as energy goes,
5859                                      how do you usually find a, a new restaurant *t*-1?
2931                                                           uh, who's the last one *t*-1.
7463                          oh, g-, where *ich*-3 have i gone *t*-1 that *t*-2's anywhere.
2328        you know, how can you help *-1 depending on the day care center *t*-2, you know,
6779                                                  uh, how do you feel about taxes *t*-1?
3938                                                                  where do you go *t*-1?
Name: EntireSentence, dtype: object

In [24]:
sample2 = df_final.sample(2, random_state=33)

In [25]:
sample2.PreceedingContext

5347    ###that's all mexico, i think 0 *t*-1.###speakerb57.###yes,###uh-huh.### and, but it was wonderful.###so we really like those.###and, uh, i guess 0 we just like *-1 to go someplace, someplace entirely different.###we've been to a few places in this country.###we still have a lot 0 *-1 to see *t*-2, though.###speakera58.
7270                                                                                                              ###i mean, pretty much it's a white shirt and slacks and shoes and a coat.###speakerb12.###uh-huh.###speakera13.###speakerb14.###uh-huh.###speakera15.###speakerb16.###speakera17.###but, it was very different before.
Name: PreceedingContext, dtype: object

In [26]:
sample2.EntireSentence

5347    where are you from *t*-1 originally?
7270                 how do you dress *t*-1?
Name: EntireSentence, dtype: object

In [27]:
pilot = pilot.drop([5928,2328], axis=0)

In [32]:
pilot2 = pd.concat([pilot, sample2])

In [36]:
# remove the someresponse column
pilot2 = pilot2.drop(columns="SomeResponse")

In [37]:
pilot2.head(1)

,TGrepID,EntireSentence,PreceedingContext,AResponse,AllResponse,TheResponse
3582,61578:4,how are your kids handling it *t*-1?,"###speakera161.###no,### i, i know,###i mean he's always, you know, he really does have a good home.###i think 0 that's what *t*-1 really bothers me the most about it.###speakerb162.###yeah.###speakera163.###you know, it's like,###speakerb164.",What is a way...?,What is every way...?,What is the way...?


In [38]:
pilot2.to_csv("../../experiments/clean_corpus/pilot2.txt",header=True,sep="\t",index=False)

### TEST SAMPLE

save to csv

In [30]:
def generate_stimuli(df):
    who = df[df["EntireSentence"]]

In [31]:
d.to_csv("testfile.txt",header=True,sep="\t",index=False)

NameError: name 'd' is not defined